In [1]:
# Importing dependencies
import numpy as np
import cryptoaml.datareader as cdr
from collections import OrderedDict
from cryptoaml.models import (RandomForestAlgo)

/home/delinvas/anaconda3/envs/btc-classifier/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
data = cdr.get_data("elliptic")
data_sets = data.train_test_split(train_size=0.7, feat_set=["LF", "AF"])

In [3]:
rf = RandomForestAlgo(n_estimators=50, max_features=50)
print(rf.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 50, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 50, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [ ]:
metrics = {}
total_runs = 200

for feature_set in data_sets:
    
    # Get current feature set 
    tmp_dataset = data_sets[feature_set]
    
    # Training set 
    tmp_dataset_train_X = tmp_dataset.train_X
    tmp_dataset_train_y = tmp_dataset.train_y
    
    # Test set 
    tmp_dataset_test_X = tmp_dataset.test_X
    tmp_dataset_test_y = tmp_dataset.test_y
    
    metrics[feature_set] = {}
    metrics[feature_set]["precision"] = []
    metrics[feature_set]["recall"] = []
    metrics[feature_set]["f1"] = []
    metrics[feature_set]["f1_micro"] = []
    
    for i in range(total_runs):
        rf.fit(tmp_dataset_train_X, tmp_dataset_train_y)
        tmp_metrics = rf.evaluate(metrics=["precision", "recall", "f1", "f1_micro"], 
                                  X=tmp_dataset_test_X,
                                  y=tmp_dataset_test_y)
        
        metrics[feature_set]["precision"].append(tmp_metrics["precision"])
        metrics[feature_set]["recall"].append(tmp_metrics["recall"])
        metrics[feature_set]["f1"].append(tmp_metrics["f1"])
        metrics[feature_set]["f1_micro"].append(tmp_metrics["f1_micro"])

In [ ]:
for feature_set in metrics:
    print("===========================================")
    print(feature_set)
    print("===========================================")

    for metric in metrics[feature_set]:
        metric_mean = round(np.mean(metrics[feature_set][metric]),3)
        metric_std = round(np.std(metrics[feature_set][metric]),3)
        print("\t {}\n\t\tmean:{}\n\t\tstd:{}".format(metric, metric_mean, metric_std))